In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()
# Install all your required packages here using mamba
!mamba install -q scikit-learn graph-tool

✨🍰✨ Everything looks OK!


In [2]:
import torch
from torch.utils.data import Subset
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt

# Define the transformation pipeline:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: (x > 0.5).float()),  # Binarize the image
    transforms.Lambda(lambda x: x.view(-1))           # Flatten into a 784-dim vector
])

# Load the training set (set download=True if running for the first time)
mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Get indices for images where the label is 1
indices = (mnist_train.targets == 1).nonzero().squeeze()

# Create a subset containing only the '1's
mnist_train_ones = Subset(mnist_train, indices)

print(f"Total number of '1' images in the training set: {len(mnist_train_ones)}")

# Stack all 784-dim vectors from the filtered dataset
all_vectors = torch.stack([img for img, _ in mnist_train_ones])
unique_vectors = torch.unique(all_vectors, dim=0)

print(f"Total images in mnist_train_ones: {all_vectors.shape[0]}")
print(f"Unique images: {unique_vectors.shape[0]}")

if all_vectors.shape[0] == unique_vectors.shape[0]:
    print("All 784-dimensional vectors are unique.")
else:
    print("There are duplicates in the 784-dimensional vectors.")



Total number of '1' images in the training set: 6742
Total images in mnist_train_ones: 6742
Unique images: 6726
There are duplicates in the 784-dimensional vectors.


In [3]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import graph_tool as gt

def visualize_threshold_graph_gt_weighted(unique_vectors, distance_threshold):
    """
    Builds a weighted graph with graph-tool where each unique vector is a node and an edge is added
    between nodes if their Euclidean distance is <= distance_threshold.
    The edge is weighted by the real Euclidean distance between the nodes.

    Args:
        unique_vectors (torch.Tensor or np.array): Array/tensor of shape (n, 784) containing unique images.
        distance_threshold (float): Maximum Euclidean distance for adding an edge between two nodes.

    Returns:
        g (graph_tool.Graph): The constructed weighted graph.
    """
    # Convert tensor to numpy array if necessary.
    X = unique_vectors

    n = X.shape[0]
    print(f"Building weighted graph for {n} nodes...")

    # Use NearestNeighbors to find all pairs within the distance threshold.
    nbrs = NearestNeighbors(radius=distance_threshold, algorithm='auto').fit(X)
    distances, indices = nbrs.radius_neighbors(X)

    # Create an undirected graph using graph-tool.
    g = gt.Graph(directed=False)
    g.add_vertex(n)  # Add n vertices

    # Build a list of edges with weights
    edge_list = []
    for i, (dists, neigh) in enumerate(zip(distances, indices)):
        # For each neighbor j of node i, add edge only if i < j to avoid duplicates.
        edge_list.extend([[i, j, d] for d, j in zip(dists, neigh) if i < j])

    # Convert the list of edges to a NumPy array (with float type to hold the weight)
    edge_array = np.array(edge_list)

    eweight = g.new_ep("float")
    # Add all edges at once
    g.add_edge_list(edge_array, eprops=[eweight])

    print(f"Weighted graph built with {g.num_edges()} edges.")
    return g

# Example usage:
g = visualize_threshold_graph_gt_weighted(unique_vectors, distance_threshold=6)


Building weighted graph for 6726 nodes...
Weighted graph built with 4681902 edges.


In [ ]:
import numpy as np
import graph_tool.all as gt
import graph_tool.topology as gt_topo

def find_furthest_nodes_double_sweep_largest_component(g, X):
    """
    For a graph g (built with graph-tool) and an array X of shape (n, 784) corresponding to the
    unique vectors (with row i corresponding to vertex i), this function:
      1. Adds a vertex property to store original indices.
      2. Extracts the largest connected component.
      3. Finds two nodes that are farthest apart (approximate diameter endpoints) using a double-sweep algorithm
         on the subgraph.
      4. Computes the shortest path between them in the subgraph.
      5. Maps the subgraph vertices back to the original graph using the stored property.
      6. Extracts the corresponding 784-d vectors and computes delta vectors along the path.

    Returns:
      endpoints_sub: Tuple with the two endpoint indices in the subgraph (original indices).
      path_indices: List of original graph vertex indices along the shortest path.
      path_vectors: Array of the corresponding 784-d vectors for each vertex on the path.
      path_deltas: List of delta vectors between consecutive nodes on the path.
    """
    # --- Step 1: Store original vertex indices as a vertex property ---
    orig_index = g.new_vertex_property("int")
    for v in g.vertices():
        orig_index[v] = int(v)

    # --- Step 2: Extract the largest connected component ---
    comp, hist = gt.label_components(g)
    largest_comp = np.argmax(hist)
    vfilt = comp.a == largest_comp
    g_sub = gt.GraphView(g, vfilt=vfilt)

    # Build a mapping from subgraph vertex object to original graph index.
    vertex_to_orig = {v: int(orig_index[v]) for v in g_sub.vertices()}
    # Also, get a list of subgraph vertices.
    vertices_sub = list(g_sub.vertices())
    n_sub = len(vertices_sub)
    print(f"Largest connected component has {n_sub} vertices (out of {int(g.num_vertices())} total).")

    # --- Step 3: Double Sweep Algorithm on the subgraph ---
    # Use the list of vertices instead of range(n_sub)
    v0 = vertices_sub[0]
    dmap = gt.shortest_distance(g_sub, source=v0)
    # Find the vertex farthest from v0 among vertices_sub.
    farthest_idx = np.argmax([dmap[v] for v in vertices_sub])
    v1 = vertices_sub[farthest_idx]

    dmap2 = gt.shortest_distance(g_sub, source=v1)
    farthest_idx2 = np.argmax([dmap2[v] for v in vertices_sub])
    v2 = vertices_sub[farthest_idx2]

    endpoints_sub = (vertex_to_orig[v1], vertex_to_orig[v2])
    diameter_length = dmap2[v2]
    print("Diameter endpoints in subgraph (original indices):", endpoints_sub, "with length:", diameter_length)

    # --- Step 4: Compute the shortest path in the subgraph between v1 and v2 ---
    path_vertices_sub, _ = gt_topo.shortest_path(g_sub, source=v1, target=v2)
    # path_vertices_sub is a list of vertex objects in g_sub.
    print("Shortest path in subgraph (vertex objects):", list(path_vertices_sub))

    # --- Step 5: Map subgraph vertex objects back to original graph indices ---
    path_indices = [vertex_to_orig[v] for v in path_vertices_sub]
    print("Mapped original graph vertex indices for path:", path_indices)

    # --- Step 6: Extract the corresponding 784-d vectors and compute delta vectors ---
    path_vectors = X[path_indices]
    path_deltas = [path_vectors[i+1] - path_vectors[i] for i in range(len(path_vectors) - 1)]

    return endpoints_sub, path_indices, path_vectors, path_deltas

# Example usage:
# Assume unique_vectors is your NumPy array of shape (n, 784)
endpoints_sub, path_indices, path_vectors, path_deltas = find_furthest_nodes_double_sweep_largest_component(g, unique_vectors)

print("\n--- Results ---")
print("Furthest endpoints in subgraph (original indices):", endpoints_sub)
print("Path indices (original graph):", path_indices)
print("Path vectors shape:", np.array(path_vectors).shape)
print("Number of delta vectors:", len(path_deltas))


In [ ]:
import matplotlib.pyplot as plt

def plot_path_as_images(path_vectors):
    """
    Given an array of 784-d vectors (each representing an MNIST image),
    reshapes them into 28x28 images and plots them side-by-side.

    Args:
        path_vectors (np.array): Array of shape (n_steps, 784)
    """
    num_steps = len(path_vectors)
    # Create a figure with one subplot per image along the path.
    fig, axs = plt.subplots(1, num_steps, figsize=(num_steps * 2, 2))

    # If only one image, ensure axs is iterable.
    if num_steps == 1:
        axs = [axs]

    for i, vec in enumerate(path_vectors):
        img = vec.reshape(28, 28)
        axs[i].imshow(img, cmap='gray')
        axs[i].axis("off")
        axs[i].set_title(f"Step {i}", fontsize=10)

    plt.tight_layout()
    plt.show()

# Assuming path_vectors is the output from the previous function
plot_path_as_images(path_vectors)


In [ ]:
import numpy as np

# Assume path_deltas is a list of 10 arrays, each of shape (784,)
# First, stack them into a 2D array with shape (10, 784)
deltas_array = np.stack(path_deltas)  # Shape: (10, 784)

# Transpose the array to get shape (784, 10)
deltas_by_dimension = deltas_array.T  # Each row now corresponds to a dimension

# If you need a list of 784 arrays (each of length 10)
deltas_list_by_dimension = [deltas_by_dimension[i, :] for i in range(deltas_by_dimension.shape[0])]

print("Shape of deltas_array (by path step):", deltas_array.shape)
print("Shape of deltas_by_dimension (by dimension):", deltas_by_dimension.shape)


In [ ]:
import numpy as np

# Assuming deltas_by_dimension is a NumPy array of shape (784, 10)
indices_with_one = np.where(np.any(deltas_by_dimension != 0, axis=1))[0]
print("Indices with at least one 1:", indices_with_one)

import numpy as np
import pandas as pd

# Assuming deltas_by_dimension is a numpy array of shape (784, 10)
# Compute the count of 1s for each row
ones_count = np.sum(deltas_by_dimension != 0, axis=1)

# Create a DataFrame with row numbers and count of 1s
df = pd.DataFrame({
    'Row': np.arange(deltas_by_dimension.shape[0]),
    "Count of 1's": ones_count
})

# Sort the DataFrame in descending order by the count of 1s
df_sorted = df.sort_values(by="Count of 1's", ascending=False)

# Display the sorted table
print(df_sorted)


In [4]:
import numpy as np
import graph_tool.all as gt

# Label connected components; comp is a property map with component labels,
# and hist contains the sizes of each component.
comp, hist = gt.label_components(g)

# Identify the label corresponding to the largest component.
largest_component_label = np.argmax(hist)

# Create a vertex filter that is True for vertices in the largest component.
vfilt = comp.a == largest_component_label

# Create a subgraph view using the vertex filter.
g_lcc = gt.GraphView(g, vfilt=vfilt)

print("Largest connected component has", g_lcc.num_vertices(), "vertices and", g_lcc.num_edges(), "edges.")

# Get indices of nodes in the largest connected component
largest_component_indices = np.where(vfilt)[0]
print("Indices in largest component:", largest_component_indices)

# Convert indices to a torch tensor (if your unique_vectors is a torch.Tensor)
largest_component_indices_tensor = torch.tensor(largest_component_indices, dtype=torch.long)

# Filter unique_vectors to get only those in the largest component
largest_component_vectors = unique_vectors[largest_component_indices_tensor]
print(f"Using {largest_component_vectors.shape[0]} vectors for training.")

Largest connected component has 6631 vertices and 4681893 edges.
Indices in largest component: [   0    1    2 ... 6716 6723 6724]
Using 6631 vectors for training.


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt

# Create a TensorDataset using the unique vectors
unique_dataset = TensorDataset(largest_component_vectors)

# Define a simple autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self, latent_dim=32):
        super(Autoencoder, self).__init__()
        # Encoder: 784 -> 256 -> 128 -> latent_dim
        self.encoder = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim),
            nn.ReLU()
        )
        # Decoder: latent_dim -> 128 -> 256 -> 784
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Sigmoid()  # Output between 0 and 1
        )

    def forward(self, x):
        z = self.encoder(x)
        reconstructed = self.decoder(z)
        return reconstructed

    def get_latent(self, x):
        return self.encoder(x)

# Training parameters
batch_size = 256
num_epochs = 50
learning_rate = 1e-3
latent_dim = 32

# Create DataLoader for the unique dataset
unique_loader = DataLoader(unique_dataset, batch_size=batch_size, shuffle=True)

# Set up device and model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Autoencoder(latent_dim=latent_dim).to(device)

# Loss function and optimizer
criterion = nn.BCELoss()  # Use binary cross entropy since inputs are binary
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop for the autoencoder on unique vectors
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in unique_loader:
        images = batch[0].to(device)  # images shape: [batch_size, 784]
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, images)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(unique_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Extract latent representations for analysis or visualization
model.eval()
with torch.no_grad():
    sample_batch = next(iter(unique_loader))[0].to(device)
    latent_codes = model.get_latent(sample_batch)
    print("Latent representations shape:", latent_codes.shape)


Epoch [1/50], Loss: 0.4262
Epoch [2/50], Loss: 0.1269
Epoch [3/50], Loss: 0.1163
Epoch [4/50], Loss: 0.1071
Epoch [5/50], Loss: 0.0800
Epoch [6/50], Loss: 0.0692
Epoch [7/50], Loss: 0.0640
Epoch [8/50], Loss: 0.0582
Epoch [9/50], Loss: 0.0524
Epoch [10/50], Loss: 0.0507
Epoch [11/50], Loss: 0.0498
Epoch [12/50], Loss: 0.0492
Epoch [13/50], Loss: 0.0487
Epoch [14/50], Loss: 0.0485
Epoch [15/50], Loss: 0.0482
Epoch [16/50], Loss: 0.0479
Epoch [17/50], Loss: 0.0475
Epoch [18/50], Loss: 0.0469
Epoch [19/50], Loss: 0.0444
Epoch [20/50], Loss: 0.0401
Epoch [21/50], Loss: 0.0377
Epoch [22/50], Loss: 0.0366
Epoch [23/50], Loss: 0.0359
Epoch [24/50], Loss: 0.0354
Epoch [25/50], Loss: 0.0347
Epoch [26/50], Loss: 0.0341
Epoch [27/50], Loss: 0.0333
Epoch [28/50], Loss: 0.0325
Epoch [29/50], Loss: 0.0318
Epoch [30/50], Loss: 0.0312
Epoch [31/50], Loss: 0.0306
Epoch [32/50], Loss: 0.0300
Epoch [33/50], Loss: 0.0293
Epoch [34/50], Loss: 0.0282
Epoch [35/50], Loss: 0.0274
Epoch [36/50], Loss: 0.0267
E

In [23]:
import torch
import numpy as np
from torch.utils.data import DataLoader

def compare_distances(model, dataset, device, sample_size=50):
    # Randomly sample sample_size images from the dataset
    indices = torch.randperm(len(dataset))[:sample_size]
    sample = torch.stack([dataset[i][0] for i in indices]).to(device)  # shape: [sample_size, 784]

    # Get latent representation for the sample
    model.eval()
    with torch.no_grad():
        latent = model.get_latent(sample)  # shape: [sample_size, latent_dim]

    # Compute pairwise Euclidean distances in the original space and latent space
    dist_original = torch.cdist(sample, sample, p=2)  # Euclidean (L2) distance
    dist_latent = torch.cdist(latent, latent, p=2)      # Euclidean (L2) distance

    # Compute cosine similarities (convert to cosine distances if needed)
    sample_norm = sample / sample.norm(dim=1, keepdim=True)
    latent_norm = latent / latent.norm(dim=1, keepdim=True)
    cosine_sim_original = torch.mm(sample_norm, sample_norm.t())
    cosine_sim_latent = torch.mm(latent_norm, latent_norm.t())

    # For a quick quantitative comparison, compute correlation between the flattened distance matrices
    corr_euclidean = np.corrcoef(dist_original.cpu().numpy().flatten(),
                                 dist_latent.cpu().numpy().flatten())[0, 1]
    print("Correlation (Euclidean distances) between original and latent spaces:", corr_euclidean)

    corr_cosine = np.corrcoef(cosine_sim_original.cpu().numpy().flatten(),
                              cosine_sim_latent.cpu().numpy().flatten())[0, 1]
    print("Correlation (Cosine similarities) between original and latent spaces:", corr_cosine)

    return dist_original, dist_latent, cosine_sim_original, cosine_sim_latent


Correlation (Euclidean distances) between original and latent spaces: 0.9283457713573456
Correlation (Cosine similarities) between original and latent spaces: 0.8830780643562136


In [32]:
# Example usage (assuming model, unique_dataset, and device are defined as in previous code):
dist_original, dist_latent, cosine_sim_original, cosine_sim_latent = compare_distances(model, unique_dataset, device, sample_size=50)


Correlation (Euclidean distances) between original and latent spaces: 0.9172276818756147
Correlation (Cosine similarities) between original and latent spaces: 0.9008990810040167


In [5]:
import numpy as np
import torch
from torch.utils.data import TensorDataset

# First, convert the graph-tool edges to a NumPy array of indices.
edge_indices = np.array([[int(e.source()), int(e.target())] for e in g.edges()])
print(f"Total number of edges: {edge_indices.shape[0]}")

# Filter for edges where both endpoints are in the largest connected component.
# 'vfilt' is a boolean NumPy array of length n (one per unique vector)
valid_mask = vfilt[edge_indices[:, 0]] & vfilt[edge_indices[:, 1]]
valid_edges = edge_indices[valid_mask]
print(f"Number of edges in largest connected component: {valid_edges.shape[0]}")

# Convert valid_edges to a torch tensor
valid_edges_torch = torch.from_numpy(valid_edges)

Total number of edges: 4681902
Number of edges in largest connected component: 4681893


In [15]:
# Let's say you want to sample 10,000 edges from the valid ones:
num_sample_edges = 10000
total_valid_edges = valid_edges_torch.shape[0]

# Randomly sample indices from valid_edges_torch
sample_indices = torch.randperm(total_valid_edges)[:num_sample_edges]
sampled_edges = valid_edges_torch[sample_indices]

# Vectorized interpolation on the sampled edges
src_vectors = unique_vectors[sampled_edges[:, 0]]
tgt_vectors = unique_vectors[sampled_edges[:, 1]]
interpolated_tensor = (src_vectors + tgt_vectors) / 2.0
print(f"Number of interpolated vectors (sampled): {interpolated_tensor.shape[0]}")

# Create a dataset from the sampled interpolated vectors
interpolated_dataset = TensorDataset(interpolated_tensor)

Number of interpolated vectors (sampled): 10000


In [46]:
# Now use your compare_distances function on the interpolated dataset.
dist_original, dist_latent, cosine_sim_original, cosine_sim_latent = compare_distances(
    model, interpolated_dataset, device, sample_size=50
)

Correlation (Euclidean distances) between original and latent spaces: 0.9429124656331623
Correlation (Cosine similarities) between original and latent spaces: 0.933259506704428
